<a href="https://colab.research.google.com/github/cristiangodoyangel/Analisis-Sismico-Chile-Data-Science/blob/main/Terremotos_en_Chile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import plotly.express as px
import geopandas as gpd
from shapely.geometry import Point
from geopandas.tools import sjoin

In [3]:
path = "/content/drive/MyDrive/0002 Python - Data Science/Datasets/seismic_data.csv"
df_1 = pd.read_csv(path)

In [4]:
df_1.head()

,Date(UTC),Latitude,Longitude,Depth,Magnitude
0,2025-05-26 03:50:27,-19.63,-69.49,97,5.6
1,2025-05-13 00:47:58,-51.25,-72.28,28,5.1
2,2025-05-05 09:46:48,-29.49,-71.84,48,5.0
3,2025-05-05 02:17:48,-31.89,-70.88,88,5.1
4,2025-05-02 15:23:49,-27.52,-72.48,30,5.8


In [5]:
df_1

,Date(UTC),Latitude,Longitude,Depth,Magnitude
0,2025-05-26 03:50:27,-19.63,-69.49,97,5.6
1,2025-05-13 00:47:58,-51.25,-72.28,28,5.1
2,2025-05-05 09:46:48,-29.49,-71.84,48,5.0
3,2025-05-05 02:17:48,-31.89,-70.88,88,5.1
4,2025-05-02 15:23:49,-27.52,-72.48,30,5.8
...,...,...,...,...,...
4013,2012-03-24 07:28:33,-33.05,-71.06,69,5.2
4014,2012-03-19 21:52:39,-25.00,-69.75,100,5.0
4015,2012-03-10 02:26:57,-19.74,-69.25,101,5.3
4016,2012-03-04 16:27:20,-21.60,-70.06,47,5.3


In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4018 entries, 0 to 4017
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date(UTC)  4018 non-null   object 
 1   Latitude   4018 non-null   float64
 2   Longitude  4018 non-null   float64
 3   Depth      4018 non-null   int64  
 4   Magnitude  4018 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 157.1+ KB


In [7]:
# Convertimos la fecha original a datetime
df_1['Fecha'] = pd.to_datetime(df_1['Date(UTC)'])

# Convertimos a horario chileno
df_1['Fecha_Chile'] = df_1['Fecha'].dt.tz_localize('UTC').dt.tz_convert('America/Santiago')

# Extraemos hora y mes
df_1['Hora'] = df_1['Fecha_Chile'].dt.hour
df_1['Mes'] = df_1['Fecha_Chile'].dt.month

# Diccionario de meses en español
meses = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}
df_1['Mes_Nombre'] = df_1['Mes'].map(meses)
df_1.describe().T

,count,mean,min,25%,50%,75%,max,std
Latitude,4018.0,-27.837778,-62.35,-31.82,-29.595,-21.5125,-11.07,5.961963
Longitude,4018.0,-70.851127,-75.73,-71.59,-71.18,-70.03,-58.39,1.293387
Depth,4018.0,61.524639,2.0,36.0,50.0,86.0,624.0,38.27661
Magnitude,4018.0,4.428945,2.3,4.0,4.4,4.8,8.4,0.688666
Fecha,4018,2019-04-08 11:26:32.820308992,2012-03-03 11:01:47,2016-07-14 17:58:09,2019-06-01 21:22:07,2021-09-02 22:38:11,2025-05-26 03:50:27,NaN
Hora,4018.0,11.336237,0.0,5.0,11.0,17.0,23.0,6.905795
Mes,4018.0,6.574664,1.0,4.0,7.0,9.0,12.0,3.419187


In [8]:
df_1['Fecha'] = pd.to_datetime(df_1['Date(UTC)'])
df_1['Fecha_Chile'] = df_1['Fecha'].dt.tz_localize('UTC').dt.tz_convert('America/Santiago')
df_1['Hora'] = df_1['Fecha_Chile'].dt.hour
df_1['Mes'] = df_1['Fecha_Chile'].dt.month

# Diccionario de meses
meses = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}
df_1['Mes_Nombre'] = df_1['Mes'].map(meses)

# Orden correcto de meses
orden_meses = list(meses.values())
df_1['Mes_Nombre'] = pd.Categorical(df_1['Mes_Nombre'], categories=orden_meses, ordered=True)

# Agrupamos por mes y hora
pivot_data = df_1.groupby(['Mes_Nombre', 'Hora']).size().reset_index(name='Cantidad')

fig = px.density_heatmap(
    pivot_data,
    x='Hora',
    y='Mes_Nombre',
    z='Cantidad',
    color_continuous_scale='Blues',
    title='🌡️ Intensidad de Sismos por Mes y Hora en Chile (2012–2025)',
    labels={'Hora': 'Hora del Día', 'Mes_Nombre': 'Mes', 'Cantidad': 'N° de Sismos'}
)

fig.update_layout(
    xaxis=dict(dtick=1, title_font_size=16, tickfont_size=14),
    yaxis=dict(title_font_size=16, tickfont_size=14),
    title_font_size=20,
    width=950,
    height=600
)

fig.show()





/tmp/ipython-input-8-3183836088.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pivot_data = df_1.groupby(['Mes_Nombre', 'Hora']).size().reset_index(name='Cantidad')


In [ ]:

orden_meses = ["Enero", "Febrero", "Marzo", "Abril",
               "Mayo", "Junio", "Julio", "Agosto",
               "Septiembre", "Octubre", "Noviembre", "Diciembre"]

# Extraemos año desde la fecha
df_1['Año'] = df_1['Fecha_Chile'].dt.year

# Agrupamos por Año y Mes
conteo_mensual = df_1.groupby(['Año', 'Mes_Nombre']).size().reset_index(name='Cantidad')

# Calculamos el promedio mensual
promedio_por_mes = conteo_mensual.groupby('Mes_Nombre')['Cantidad'].mean().reset_index()
promedio_por_mes['Cantidad'] = promedio_por_mes['Cantidad'].round(1)

# Aplicamos orden categórico
promedio_por_mes['Mes_Nombre'] = pd.Categorical(
    promedio_por_mes['Mes_Nombre'],
    categories=orden_meses,
    ordered=True
)

# Filtramos los primeros 4 meses
primer_cuatrimestre = promedio_por_mes[promedio_por_mes['Mes_Nombre'].isin(["Enero", "Febrero", "Marzo", "Abril"])]

# Creamos el gráfico
fig = px.bar(
    primer_cuatrimestre,
    x='Mes_Nombre',
    y='Cantidad',
    text='Cantidad',
    title='📊 Promedio por Mes: Enero a Abril',
    labels={'Mes_Nombre': 'Mes', 'Cantidad': 'Promedio de Sismos'},
    color_discrete_sequence=['#00a8ca']
)

# Personalización del gráfico
fig = px.bar(
    primer_cuatrimestre,
    x='Mes_Nombre',
    y='Cantidad',
    text='Cantidad',
    title='📊 Promedio de Sismos: Enero a Abril',
    labels={'Mes_Nombre': 'Mes', 'Cantidad': 'Promedio de Sismos'},
    color_discrete_sequence=['#00a8ca']
)

fig.update_traces(
    textposition='outside',
    textfont_size=20,
    width=0.5
)
fig.update_layout(
    title=dict(
        text='Promedio de Sismos: Enero a Abril',
        x=0.5,
        font=dict(size=30, family='Arial Black', color='black')
    ),
    yaxis_title=dict(
        text='Promedio de Sismos',
        font=dict(size=22, family='Arial Black', color='black')
    ),
    xaxis=dict(
        title=dict(
            text='Mes',
            font=dict(size=22, family='Arial Black', color='black')
        ),
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    yaxis=dict(
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    height=650,
    width=800,
    margin=dict(t=80, b=60, l=60, r=60),
    showlegend=False,
    bargap=0.05
)


fig.show()



/tmp/ipython-input-22-832613037.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipython-input-22-832613037.py:12: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
# Filtramos los otros 4 meses
primer_cuatrimestre = promedio_por_mes[promedio_por_mes['Mes_Nombre'].isin(["Mayo", "Junio", "Julio", "Agosto"])]

# Creamos el gráfico
fig = px.bar(
    primer_cuatrimestre,
    x='Mes_Nombre',
    y='Cantidad',
    text='Cantidad',
    title='Promedio de Sismos: Mayo a Agosto',
    labels={'Mes_Nombre': 'Mes', 'Cantidad': 'Promedio de Sismos'},
    color_discrete_sequence=['#00a8ca']
)

# Personalización del gráfico
fig = px.bar(
    primer_cuatrimestre,
    x='Mes_Nombre',
    y='Cantidad',
    text='Cantidad',
    title='Promedio de Sismos: Mayo a Agosto',
    labels={'Mes_Nombre': 'Mes', 'Cantidad': 'Promedio de Sismos'},
    color_discrete_sequence=['#00a8ca']
)

fig.update_traces(
    textposition='outside',
    textfont_size=20,
    width=0.5
)
fig.update_layout(
    title=dict(
        text='Promedio de Sismos: Mayo a Agosto',
        x=0.5,
        font=dict(size=30, family='Arial Black', color='black')
    ),
    yaxis_title=dict(
        text='Promedio de Sismos',
        font=dict(size=22, family='Arial Black', color='black')
    ),
    xaxis=dict(
        title=dict(
            text='Mes',
            font=dict(size=22, family='Arial Black', color='black')
        ),
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    yaxis=dict(
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    height=650,
    width=800,
    margin=dict(t=80, b=60, l=60, r=60),
    showlegend=False,
    bargap=0.05
)


fig.show()


In [ ]:
# Filtramos los últimos 4 meses
primer_cuatrimestre = promedio_por_mes[promedio_por_mes['Mes_Nombre'].isin(["Septiembre", "Octubre", "Noviembre", "Diciembre"])]

# Creamos el gráfico
fig = px.bar(
    primer_cuatrimestre,
    x='Mes_Nombre',
    y='Cantidad',
    text='Cantidad',
    title='Promedio de Sismos: Septiembre a Diciembre',
    labels={'Mes_Nombre': 'Mes', 'Cantidad': 'Promedio de Sismos'},
    color_discrete_sequence=['#00a8ca']
)

# Personalización del gráfico
fig = px.bar(
    primer_cuatrimestre,
    x='Mes_Nombre',
    y='Cantidad',
    text='Cantidad',
    title='Promedio de Sismos: Septiembre a Diciembre',
    labels={'Mes_Nombre': 'Mes', 'Cantidad': 'Promedio de Sismos'},
    color_discrete_sequence=['#00a8ca']
)

fig.update_traces(
    textposition='outside',
    textfont_size=20,
    width=0.5
)
fig.update_layout(
    title=dict(
        text='Promedio de Sismos: Septiembre a Diciembre',
        x=0.5,
        font=dict(size=30, family='Arial Black', color='black')
    ),
    yaxis_title=dict(
        text='',
        font=dict(size=22, family='Arial Black', color='black')
    ),
    xaxis=dict(
        title=dict(
            text='',
            font=dict(size=22, family='Arial Black', color='black')
        ),
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    yaxis=dict(
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    height=650,
    width=800,
    margin=dict(t=80, b=60, l=60, r=60),
    showlegend=False,
    bargap=0.05
)


fig.show()


In [ ]:

# Gráfico Total de Sismos por Zona de Chile
fig = px.bar(
    sismos_por_zona,
    x='Zona',
    y='Cantidad',
    text='Cantidad',
    title='Total de Sismos por Zona de Chile',
    labels={'Zona': 'Zona', 'Cantidad': 'Cantidad de Sismos'},
    color='Zona',
    color_discrete_map={'Norte': '#00a8ca', 'Centro': '#00a8ca', 'Sur': '#00a8ca'}
)

fig.update_traces(
    textposition='outside',
    textfont_size=20,
    width=0.5
)

fig.update_layout(
    title=dict(
        text='Total de Sismos por Zona de Chile',
        x=0.5,
        font=dict(size=30, family='Arial Black', color='black')
    ),
    yaxis_title=dict(
        text='',
        font=dict(size=22, family='Arial Black', color='black')
    ),
    xaxis=dict(
        title=dict(
            text='',
            font=dict(size=22, family='Arial Black', color='black')
        ),
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    yaxis=dict(
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    height=650,
    width=800,
    margin=dict(t=80, b=60, l=60, r=60),
    showlegend=False,
    bargap=0.05
)

fig.show()


In [ ]:

# Aplicar la zona (¡IMPORTANTE correr esto antes del top3!)
df_1['Zona'] = df_1['Latitude'].apply(clasificar_zona)


# Top 3 sismos por magnitud
top3 = df_1.nlargest(3, 'Magnitude')[['Fecha_Chile', 'Latitude', 'Longitude', 'Magnitude', 'Zona']].reset_index(drop=True)


# Gráfico en Plotly Express
fig = px.bar(
    top3,
    x='Zona',
    y='Magnitude',
    text='Magnitude',
    labels={'Zona': '', 'Magnitude': ''},
    color='Zona',
    color_discrete_map={'Norte': '#00a9cb', 'Centro': '#00a9cb', 'Sur': '#00a9cb'}
)

# Estilo uniforme para todos los gráficos
fig.update_traces(
    textposition='outside',
    textfont_size=20,
    width=0.5
)

fig.update_layout(
    title=dict(
        text='Sismos Más Fuertes por Zona',
        x=0.5,
        font=dict(size=30, family='Arial Black', color='black')
    ),
    yaxis_title=dict(
        text='',
        font=dict(size=22, family='Arial Black', color='black')
    ),
    xaxis=dict(
        title=dict(
            text='',
            font=dict(size=22, family='Arial Black', color='black')
        ),
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    yaxis=dict(
        tickfont=dict(size=20, family='Arial Black', color='black'),
        range=[0, top3['Magnitude'].max() + 1],
        ticklabelposition="outside",  # asegura que estén fuera del eje
        ticklabeloverflow="allow",   # permite que no se corte el texto
        ticklabelstep=1,
        ticklabelstandoff=10         # <-- este es el que separa los números del eje

    ),
    height=650,
    width=800,
    margin=dict(t=80, b=60, l=60, r=60),
    showlegend=False,
    bargap=0.05
)

fig.show()





In [ ]:
# Exportar el DF
df_1.to_csv('df_1_sismos_chile.csv', index=False)

from google.colab import files
files.download('df_1_sismos_chile.csv')  # o .xlsx



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Sismos por Región de Chile

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Crear la geometría (puntos con lat/lon)
df_1['geometry'] = df_1.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf_sismos = gpd.GeoDataFrame(df_1, geometry='geometry', crs="EPSG:4326")


In [ ]:
# geojson de regiones de Chile
url = "https://raw.githubusercontent.com/caracena/chile-geojson/master/regiones.json"


# se cargan las regiones
gdf_regiones = gpd.read_file(url)


In [ ]:
from geopandas.tools import sjoin

# Unimos los sismos con las regiones
gdf_sismos_region = gpd.sjoin(gdf_sismos, gdf_regiones, how="inner", predicate="within")

# Revisamos las columnas disponibles
gdf_sismos_region[['Fecha_Chile', 'Magnitude', 'Region']]


,Fecha_Chile,Magnitude,Region
0,2025-05-25 23:50:27-04:00,5.6,Región de Tarapacá
1,2025-05-12 20:47:58-04:00,5.1,Región de Magallanes y Antártica Chilena
3,2025-05-04 22:17:48-04:00,5.1,Región de Coquimbo
7,2025-04-17 20:10:18-04:00,6.0,Región de Antofagasta
8,2025-04-08 18:30:45-04:00,5.2,Región de Tarapacá
...,...,...,...
4013,2012-03-24 04:28:33-03:00,5.2,Región de Valparaíso
4014,2012-03-19 18:52:39-03:00,5.0,Región de Antofagasta
4015,2012-03-09 23:26:57-03:00,5.3,Región de Tarapacá
4016,2012-03-04 13:27:20-03:00,5.3,Región de Antofagasta


In [ ]:
# Agrupar por región y obtener top 3 por magnitud
top_3_por_region = (
    gdf_sismos_region.sort_values(by='Magnitude', ascending=False)
    .groupby('Region')
    .head(3)
    .sort_values(['Region', 'Magnitude'], ascending=[True, False])
    .reset_index(drop=True)
)

# Mostrar
top_3_por_region.head(20)  # Muestra los primeros 20 resultados


,Date(UTC),Latitude,Longitude,Depth,Magnitude,Fecha,Fecha_Chile,Hora,Mes,Mes_Nombre,...,Zona,geometry,index_right,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,Region
0,2021-01-24 00:07:44,-33.35,-70.21,123,5.7,2021-01-24 00:07:44,2021-01-23 21:07:44-03:00,21,1,Enero,...,Centro,POINT (-70.21 -33.35),8,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,Región Metropolitana de Santiago
1,2023-07-09 17:11:33,-33.23,-70.27,111,5.6,2023-07-09 17:11:33,2023-07-09 13:11:33-04:00,13,7,Julio,...,Centro,POINT (-70.27 -33.23),8,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,Región Metropolitana de Santiago
2,2023-03-21 17:38:13,-33.61,-71.33,64,5.5,2023-03-21 17:38:13,2023-03-21 14:38:13-03:00,14,3,Marzo,...,Centro,POINT (-71.33 -33.61),8,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,Región Metropolitana de Santiago
3,2024-07-19 01:50:47,-23.12,-68.07,145,7.4,2024-07-19 01:50:47,2024-07-18 21:50:47-04:00,21,7,Julio,...,Norte,POINT (-68.07 -23.12),2,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,Región de Antofagasta
4,2020-06-03 07:35:34,-23.25,-68.53,123,6.9,2020-06-03 07:35:34,2020-06-03 03:35:34-04:00,3,6,Junio,...,Norte,POINT (-68.53 -23.25),2,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,Región de Antofagasta
5,2022-05-10 23:06:29,-23.38,-67.31,250,6.9,2022-05-10 23:06:29,2022-05-10 19:06:29-04:00,19,5,Mayo,...,Norte,POINT (-67.31 -23.38),2,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,Región de Antofagasta
6,2015-03-23 04:51:38,-18.42,-69.27,121,6.3,2015-03-23 04:51:38,2015-03-23 01:51:38-03:00,1,3,Marzo,...,Norte,POINT (-69.27 -18.42),0,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,Región de Arica y Parinacota
7,2017-10-10 06:32:20,-18.49,-69.74,100,6.3,2017-10-10 06:32:20,2017-10-10 03:32:20-03:00,3,10,Octubre,...,Norte,POINT (-69.74 -18.49),0,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,Región de Arica y Parinacota
8,2018-01-21 01:06:40,-18.88,-69.61,129,6.2,2018-01-21 01:06:40,2018-01-20 22:06:40-03:00,22,1,Enero,...,Norte,POINT (-69.61 -18.88),0,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,Región de Arica y Parinacota
9,2013-01-30 20:15:41,-28.18,-70.88,52,6.7,2013-01-30 20:15:41,2013-01-30 17:15:41-03:00,17,1,Enero,...,Centro,POINT (-70.88 -28.18),5,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,Región de Atacama


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


# geometría para geopandas
df_1['geometry'] = df_1.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf_sismos = gpd.GeoDataFrame(df_1, geometry='geometry', crs="EPSG:4326")


# Unión espacial: sismo dentro de región
gdf_sismos_region = gpd.sjoin(gdf_sismos, gdf_regiones, how="inner", predicate="within")

# Revisar columnas útiles
gdf_sismos_region[['Fecha_Chile', 'Magnitude', 'Region']].head()

# Exportar a CSV
gdf_sismos_region.to_csv("df_sismos_con_regiones.csv", index=False)

In [ ]:
from google.colab import files
files.download("df_sismos_con_regiones.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import plotly.express as px

# Cargar archivo
path = "/content/df_sismos_con_regiones.csv"
df = pd.read_csv(path)

# Convertir fecha y extraer año
df['Fecha_Chile'] = pd.to_datetime(df['Fecha_Chile'], utc=True).dt.tz_convert('America/Santiago')
df['Año'] = df['Fecha_Chile'].dt.year

# Diccionario con TODAS las variantes de nombres regionales
region_numerada = {
    'Región de Arica y Parinacota': 'XV',
    'Región de Tarapacá': 'I',
    'Región de Antofagasta': 'II',
    'Región de Atacama': 'III',
    'Región de Coquimbo': 'IV',
    'Región de Valparaíso': 'V',
    'Región Metropolitana de Santiago': 'RM',
    'Región del Libertador General Bernardo O’Higgins': 'VI',
    "Región del Libertador Bernardo O'Higgins": 'VI',
    'Región del Maule': 'VII',
    'Región de Ñuble': 'XVI',
    'Región del Biobío': 'VIII',
    'Región del Bío-Bío': 'VIII',
    'Región de La Araucanía': 'IX',
    'Región de Los Ríos': 'XIV',
    'Región de Los Lagos': 'X',
    'Región de Aysén del General Carlos Ibáñez del Campo': 'XI',
    'Región de Magallanes y Antártica Chilena': 'XII'
}

# Reemplazar nombres por numeración romana
df['Region'] = df['Region'].replace(region_numerada)

# Agrupar por Región y Año
conteo_anual = df.groupby(['Region', 'Año']).size().reset_index(name='Cantidad')

# Calcular promedio anual por Región
promedio_por_region = conteo_anual.groupby('Region')['Cantidad'].mean().round(1).reset_index()
promedio_por_region = promedio_por_region.sort_values('Region').reset_index(drop=True)
subset = promedio_por_region.iloc[0:4]

fig = px.bar(
    subset,
    x='Region',
    y='Cantidad',
    text='Cantidad',
    title='Promedio Anual de Sismos por Región'
)

fig.update_traces(
    textposition='outside',
    textfont_size=20,
    width=0.5
)

fig.update_layout(
    title=dict(
        text='Promedio Anual de Sismos por Región',
        x=0.5,
        font=dict(size=30, family='Arial Black', color='black')
    ),
    yaxis_title=dict(
        text='',
        font=dict(size=22, family='Arial Black', color='black')
    ),
    xaxis=dict(
        title=dict(
            text='',
            font=dict(size=22, family='Arial Black', color='black')
        ),
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    yaxis=dict(
        tickfont=dict(size=20, family='Arial Black', color='black'),
        range=[0, 70],
        ticklabelposition="outside",
        ticklabeloverflow="allow",
        automargin=True
    ),
    height=750,
    width=800,
    margin=dict(t=80, b=60, l=60, r=60),
    showlegend=False,
    bargap=0.05
)

fig.show()


In [ ]:
subset = promedio_por_region.iloc[4:8]

fig = px.bar(
    subset,
    x='Region',
    y='Cantidad',
    text='Cantidad',
    title='Promedio Anual de Sismos por Región'
)

fig.update_traces(
    textposition='outside',
    textfont_size=20,
    width=0.5
)

fig.update_layout(
    title=dict(
        text='Promedio Anual de Sismos por Región',
        x=0.5,
        font=dict(size=30, family='Arial Black', color='black')
    ),
    yaxis_title=dict(
        text='',
        font=dict(size=22, family='Arial Black', color='black')
    ),
    xaxis=dict(
        title=dict(
            text='',
            font=dict(size=22, family='Arial Black', color='black')
        ),
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    yaxis=dict(
        tickfont=dict(size=20, family='Arial Black', color='black'),
        range=[0, 10],
        ticklabelposition="outside",
        ticklabeloverflow="allow",
        automargin=True
    ),
    height=750,
    width=800,
    margin=dict(t=80, b=60, l=60, r=60),
    showlegend=False,
    bargap=0.05
)

fig.show()



In [ ]:
subset = promedio_por_region.iloc[8:12]

fig = px.bar(
    subset,
    x='Region',
    y='Cantidad',
    text='Cantidad',
    title='Promedio Anual de Sismos por Región'
)

fig.update_traces(
    textposition='outside',
    textfont_size=20,
    width=0.5
)

fig.update_layout(
    title=dict(
        text='Promedio Anual de Sismos por Región',
        x=0.5,
        font=dict(size=30, family='Arial Black', color='black')
    ),
    yaxis_title=dict(
        text='',
        font=dict(size=22, family='Arial Black', color='black')
    ),
    xaxis=dict(
        title=dict(
            text='',
            font=dict(size=22, family='Arial Black', color='black')
        ),
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    yaxis=dict(
        tickfont=dict(size=20, family='Arial Black', color='black'),
        range=[0, 10],
        ticklabelposition="outside",
        ticklabeloverflow="allow",
        automargin=True
    ),
    height=750,
    width=800,
    margin=dict(t=80, b=60, l=60, r=60),
    showlegend=False,
    bargap=0.05
)

fig.show()



In [ ]:
subset = promedio_por_region.iloc[12:16]

fig = px.bar(
    subset,
    x='Region',
    y='Cantidad',
    text='Cantidad',
    title='Promedio Anual de Sismos por Región'
)

fig.update_traces(
    textposition='outside',
    textfont_size=20,
    width=0.5
)

fig.update_layout(
    title=dict(
        text='Promedio Anual de Sismos por Región',
        x=0.5,
        font=dict(size=30, family='Arial Black', color='black')
    ),
    yaxis_title=dict(
        text='',
        font=dict(size=22, family='Arial Black', color='black')
    ),
    xaxis=dict(
        title=dict(
            text='',
            font=dict(size=22, family='Arial Black', color='black')
        ),
        tickfont=dict(size=20, family='Arial Black', color='black')
    ),
    yaxis=dict(
        tickfont=dict(size=20, family='Arial Black', color='black'),
        range=[0, 10],
        ticklabelposition="outside",
        ticklabeloverflow="allow",
        automargin=True
    ),
    height=750,
    width=800,
    margin=dict(t=80, b=60, l=60, r=60),
    showlegend=False,
    bargap=0.05
)

fig.show()

